In [116]:
import json
import networkx as nx
import gurobipy as gp
from gurobipy import GRB


# Open and read the JSON file
with open('Salt_Lake_Soloist.json', 'r') as file:
    data = json.load(file)


#Donor compatability dictionary. Referenced in the edge creation process
donor_recipient_compatability = {'O': ['O', 'A', 'B', 'AB'], 'A': ['A', 'AB'], 'B':['B', 'AB'], 'AB':['AB']}


#data is stored as a list (in []) per recipient/donor pair that are dictionary (key:value) and the donor value is a list ([])

print('Number of data points in file:',len(data)) #gives total amount of objects in the JSON file

Number of data points in file: 729


In [117]:
# Create a graph with nodes and edges
G = nx.DiGraph()

# Create nodes (starting points)
counter = 0
for element in data:
    G.add_nodes_from([(counter, element)])
    counter += 1

# print('Number of Donors:',len(G.nodes))                               #Quick check for node creation.


# Create edges (relations)
for node in G.nodes:
    donors = G.nodes[node]['Donor']                                     #separates donor info from the recipient info
   
    #print(donors)                                                      #Quick check for donor output
    
    for donor in donors:
        for vertex in G.nodes:                                          #This loop is creating a new variable "vertex" from the set of G.nodes based on the conditions below. The vertex is the head of the directional arrow.
            if node == vertex: 
                continue                                                #if the vertex is equal to the starting point (node), skip the code below and go to the next one(continue)
            recipient = G.nodes[vertex]['Recipient']                    #grabs the recipient value of the vertex in question. This allows the next lines to check if the recipient is compatible with the donor.
            if recipient in donor_recipient_compatability[donor]:       #if the recipient (vertex) is compatible with donor (node)
                G.add_edge(node, vertex)                                #adds edge from donor (node) to recipient (vertex)
                
print('Number of compatible pairs:', len(G.edges)) #quick check for number of edges created. Output = 181532

Number of compatible pairs: 181532


In [ ]:
# #<----------------------Graphing the data. Note extremely large data set that could take forever.-------------------------->

# # Visualize the graph, for fun
# import matplotlib.pyplot as plt

# plt.figure(figsize=(50,50)) #increases the size of the figure
# pos = nx.circular_layout(G)
# nx.draw(G, pos, with_labels = True,node_size=10000, font_size=60)

# plt.show()
# #<----------------------Graphing the data. Not required with extremely large data set.-------------------------->

In [118]:
# Finding cycles of length 2

cycle_2 = []                                       #an empty list that will have values added after running the loop below

for (i,j) in G.edges:                              #creates two variables i and j from within the edges. i being the node and j being the vertex
    G.edges[(i,j)]["visited"] = False              #initializes all edges as "not visited".
    
for (i,j) in G.edges:                              #creates two variables i and j from within the edges. i being the node and j being the vertex
    if G.edges[(i,j)]["visited"] == True: 
        continue                                   #if an edge has already been checked, skip and continue the iteration
    if (j,i) in G.edges:                           #for every edge (i,j), if there is also an edge from (j,i) 
        cycle_2.append((i,j))                      #adds cycle to list in the form i,j which shows the loop. 
        G.edges[(j,i)]["visited"] = True           #marks edge as visited so it is not checked again. 
        G.edges[(i,j)]["visited"] = True           #marks edge as visited so it is not checked again.
            
print("Total cycles of length 2:", len(cycle_2))   #prints the total number of cycles of 2 generated from the loop above. The loop prevents any duplicates. (0,1,0) and (1,0,1) are the same cycle

Total cycles of length 2: 34221


In [111]:
import gurobipy as gp
from gurobipy import GRB

# Create model object
m = gp.Model()

# Create a binary variable for each edge in cycle_2
x = m.addVars(cycle_2, vtype=GRB.BINARY)

# Objective function: maximize the number of 2-cycles selected
m.setObjective(gp.quicksum(x[e] for e in cycle_2), GRB.MAXIMIZE)

# Constraint to ensure each node is part of at most one 2-cycle
for v in G.nodes:
    m.addConstr(gp.quicksum(x[(i,j)] for (i,j) in cycle_2 if v in (i,j)) <=1) #only one constraint since we already isolated the cycles of 2 from all the edges created

# Optimize the model
m.optimize()

# Extract the optimal edges in cycles of length 2 or note that no optimal soulution was found. Needed a check step to ensure optimization was working or not.
if m.status == GRB.OPTIMAL:
    selected_edges = [e for e in cycle_2 if x[e].x > 0.5]
    print("Optimal edges in cycles of length 2:", selected_edges)
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 729 rows, 34221 columns and 68442 nonzeros
Model fingerprint: 0x3c00fdf7
Variable types: 0 continuous, 34221 integer (34221 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 183.0000000
Presolve removed 359 rows and 0 columns
Presolve time: 0.21s
Presolved: 370 rows, 34221 columns, 68442 nonzeros
Variable types: 0 continuous, 34221 integer (34221 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity     -3.6900000e+02      0s
       1        474  -1.0000000e+01  -1.875

In [ ]:
# #<---------------------- Visualize the cycles of length 2 in a circular graph not needed for large data set------------>
# H = nx.DiGraph()
# H.add_nodes_from(G.nodes)

# for (i,j) in selected_edges:
#     H.add_edge(i,j)
#     H.add_edge(j,i)

# plt.figure(figsize=(10, 10))
# pos = nx.circular_layout(H)
# # edge_colors = ['red' for (u, v) in selected_edges]

# nx.draw(H, pos, with_labels=True, node_color='skyblue', node_size=500,edge_color='gray', font_size=10, font_weight='bold', width=2, connectionstyle="arc3,rad=0.1")

# plt.title("Optimized Cycles of Length 2")
# plt.show()
# #<---------------------- Visualize the cycles of length 2 in a circular graph not needed for large data set------------>

In [24]:
#Finding Cycles of 3

cycle_3 = []                                                                   # An empty list to store cycles of length 3

# Initialize all edges as "not visited"
for (i, j) in G.edges:
    G.edges[(i, j)]["visited"] = False

# Detect cycles of length 3
for (i, j) in G.edges:
    if G.edges[(i, j)]["visited"]:
        continue                                                               # Skip if already visited
    for k in G.successors(j):                                                  # Iterate over successors of j
        if k == i or G.edges[(j, k)]["visited"]:                               # Checks if node k is equal to node i or edge j,k has already been visited
            continue                                                           # Skip if k is i or edge (j, k) is visited
        if (k, i) in G.edges and not G.edges[(k, i)]["visited"]:               # If edge k,i exists and has not already been visited
            cycle_3.append((i, j, k))  # Add the cycle (i, j, k)               # add cycle of 3 to the empty list
            G.edges[(i, j)]["visited"] = True                                  # update edge i,j to visited
            G.edges[(j, k)]["visited"] = True                                  # update edge j,k to visited
            G.edges[(k, i)]["visited"] = True                                  # update edge k,i to visited

print("Total possible cycles of length 3:", len(cycle_3))                      # prints the total possible cycles

Cycles of length 3: 321


In [112]:
#Optimization for Cycles of 3

import gurobipy as gp
from gurobipy import GRB

# Create model object
m = gp.Model()

# Create a binary variable for each edge in cycle_3
x = m.addVars(cycle_3, vtype=GRB.BINARY)

# Objective function: maximize the number of 3-cycles selected
m.setObjective(gp.quicksum(x[e] for e in cycle_3), GRB.MAXIMIZE)

# Constraints to ensure each node is part of at most one 3-cycle since we have already isolated the possible cycles of 3 in previous steps
for v in G.nodes:
    m.addConstr(gp.quicksum(x[(i,j,k)] for (i,j,k) in cycle_3 if v in (i,j,k)) <=1)

# Optimize the model
m.optimize()

# Extract the optimal edges in cycles of length 2. A check to ensure answer is optimalor infeasible
if m.status == GRB.OPTIMAL:
    selected_edges = [e for e in cycle_3 if x[e].x > 0.5]
    print("Optimal cycles of length 3:", selected_edges)
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 729 rows, 321 columns and 963 nonzeros
Model fingerprint: 0x6f17751f
Variable types: 0 continuous, 321 integer (321 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 11.0000000
Presolve removed 681 rows and 12 columns
Presolve time: 0.06s
Presolved: 48 rows, 309 columns, 933 nonzeros
Found heuristic solution: objective 11.0000000
Variable types: 0 continuous, 309 integer (309 binary)

Root relaxation: objective 1.400000e+01, 161 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

In [ ]:
# #<---------------------- Visualize the cycles of length 3 in a circular graph NOTE: large data set would not be easily seen------------>
# H = nx.DiGraph()
# H.add_nodes_from(G.nodes)

# for (i,j,k) in selected_edges:
#     H.add_edge(i,j)
#     H.add_edge(j,k)
#     H.add_edge(k,i)

# plt.figure(figsize=(100, 100))
# pos = nx.circular_layout(H)

# nx.draw(H, pos, with_labels=True, node_color='skyblue', node_size=500,edge_color='gray', font_size=10, font_weight='bold', width=2, connectionstyle="arc3,rad=0.01")

# plt.title("Optimized Cycles of Length 3")
# plt.show()
# #<---------------------- Visualize the cycles of length 3 in a circular graph NOTE: large data set would not be easily seen------------>